# Authorino: K8s-native Zero Trust API Security

Demo of [Authorino](https://github.com/kuadrant/authorino), K8s-native external authorization service, for [DevConf.US 2022](https://devconfus2022.sched.com/event/14Zsq/authorino-k8s-native-zero-trust-api-security), Boston (USA).

#### Authorino features covered in the demo
- API keys authentication
- OpenID Connect JWT verification
- Authorization based on Kubernetes SubjectAccessReview
- Authorization based on Authorino’s simple JSON pattern-matching authorization policies

## Requirements

- [Docker](https://docker.com)
- [Kind](https://kind.sigs.k8s.io)
- [jq](https://stedolan.github.io/jq)

## The stack

- **Kubernetes cluster**<br/>
  Started locally with [Kind](https://kind.sigs.k8s.io/).
- **News API**<br/>
  Application (REST API) to be protected with Authorino. The following HTTP endpoints are available:
  ```
  POST /{category}          Create a news article
  GET /{category}           List news articles
  GET /{category}/{id}      Read a news article
  DELETE /{category}/{id}   Delete a news article
  ```
- **[Envoy proxy](https://envoyproxy.io)**<br/>
  Deployed as sidecar of the News API, to serve the application with the [External Authorization](https://www.envoyproxy.io/docs/envoy/latest/configuration/http/http_filters/ext_authz_filter#config-http-filters-ext-authz) filter enabled and pointing to Authorino.
- **[Authorino Operator](https://github.com/kuadrant/authorino-operator)**<br/>
  Cluster-wide installation of the operator and CRDs to manage and use Authorino authorization services.
- **[Authorino](https://github.com/kuadrant/authorino)**<br/>
  The external authorization service, deployed in [`namespaced`](https://github.com/Kuadrant/authorino/blob/main/docs/architecture.md#cluster-wide-vs-namespaced-instances) reconciliation mode, in the same K8s namespace as the News API.
- **[Keycloak server](https://keycloak.org)**<br/>
  (Optional) For a more advanced use case of extending access to the News API for an external group of users managed in the the identity provider.
- **[Contour](https://projectcontour.io)**<br/>
  Kubernetes Ingress Controller based on the Envoy proxy, to handle the ingress traffic to the News API and to Keycloak.

> <br/>
> <b>Note:</b> For simplicity, in the demo all components are deployed without TLS.
> <br/>
> <br/>

## Prepare the environment

The following steps are typically performed beforehand by a cluster administrator. They are often out of the scope of the application developer's workflow.

Create the cluster:

In [ ]:
%%bash
kind create cluster --name authorino-demo --config cluster.yaml

Install Contour:

In [ ]:
!kubectl apply -f contour.yaml

Install the Authorino Operator:

In [ ]:
!kubectl apply -f https://raw.githubusercontent.com/Kuadrant/authorino-operator/main/config/deploy/manifests.yaml

> <br/>
> <b>Note:</b> In OpenShift, the Authorino Operator can alternatively be installed directly from the Red Hat OperatorHub, using <a href="https://olm.operatorframework.io">Operator Lifecycle Manager</a>.
> <br/>
> <br/>

Install Keycloak:

In [ ]:
!kubectl apply -f keycloak.yaml

## Run the demo

### 1. Deploy the News API

The News Agency API ("News API" for short) is minimal. It has no conception of authentication or authorization in its own. Whenever a request hits the API and it is a valid endpoint/operation, it serves the request. If it is a `POST` request to `/{category}`, it creates a news article under that news category. Creating an object here means storing it in memory. There is no persisted database. If it is a GET request to `/{category}`, it  serves the list of news articles in the category, as stored in memory. If it is a `GET` or `DELETE` to `/{category/(article-id}` it serves or deletes the requested object from memory, respectively.

Create the namespace:

In [ ]:
!kubectl create namespace news-api

Deploy the New Agency API in the namespace:

In [ ]:
!kubectl -n news-api apply -f news-api.yaml

At this point, the News API is running, but it is not protected.

Try the News API unprotected:

In [ ]:
!curl http://news-api.127.0.0.1.nip.io/sports

### 2. Lock down the News API

Request an instance of Authorino:

In [ ]:
!kubectl -n news-api apply -f authorino.yaml

> <br/>
> <b>Note:</b> With the Authorino Operator running, you can request instances of Authorino deployed cluster-wide (i.e. managing auth definitions across all namespaces in the Kubernetes cluster) or for a particular namespace (i.e., to protect workloads whose auth definitions are defined in the same namespaces as the corresponding Authorino instances themselves). In this demo, we are requesting an Authorino instance in the same namespace as the News API. Cluster-wide Authorino instances are typically setup by cluster administrators beforehand and therefore not part of the developer's workflow.
> <br/>
> <br/>

Add the Envoy sidecar to the News API:

In [ ]:
!kubectl -n news-api apply -f news-api-envoy.yaml

Try the News API behind Envoy:

In [ ]:
!curl http://news-api.127.0.0.1.nip.io/sports -i

### 3. Open up the News API for authenticated and authorized users

Create the AuthConfig:

In [ ]:
!kubectl -n news-api apply -f authconfig-1.yaml

Try the News API without a valid authentication key:

In [ ]:
!curl http://news-api.127.0.0.1.nip.io/sports -i

Create an API key:

In [ ]:
!kubectl -n news-api apply -f api-key-1.yaml

Try the News API with a valid API key before granting permission to the user:

In [ ]:
!curl -H 'Authorization: APIKEY ndyBzreUzF4zqDQsqSPMHkRhriEOtcRx' http://news-api.127.0.0.1.nip.io/sports -i

Grant permission to the API key user 'john' in the Kubernetes RBAC:

In [ ]:
!kubectl -n news-api apply -f role.yaml
!kubectl -n news-api apply -f rolebinding.yaml

Try the News API with a valid API key with permission granted to the user:

In [ ]:
!curl -H 'Authorization: APIKEY ndyBzreUzF4zqDQsqSPMHkRhriEOtcRx' http://news-api.127.0.0.1.nip.io/sports -i

Try the API for creating a news article:

In [ ]:
!curl -H 'Authorization: APIKEY ndyBzreUzF4zqDQsqSPMHkRhriEOtcRx' -X POST \
  -d '{"title":"Lionel Messi leaving Barcelona after ‘obstacles’ thwart contract renewal","body":"Barcelona have announced that Lionel Messi is leaving the club after “financial and structural obstacles” made it impossible to renew his contract. The forward, who has spent his whole career there, had been expected to re-sign after his deal expired in June. (By David Hytner)"}' \
  http://news-api.127.0.0.1.nip.io/sports

## Extra: Modify the auth scheme

### 4. Add an external Identity Provider and extra policies

In the [Keycloak Admin Console](http://keycloak.127.0.0.1.nip.io):
1. create a realm `devconf`;
2. add users `alice` and `bob` to the realm
    - make sure to mark only Alice's email as verified
    - set a password (`p`) to both users in the _Credentials_ tab
3. create an OpenID Connect client `demo` in the realm.

Modify the AuthConfig:

In [ ]:
!kubectl -n news-api apply -f authconfig-2.yaml

Grant permission for the Keycloak users 'alice' and 'bob' in the Kubernetes RBAC, by editing the subjects listed in the `RoleBinding`:

In [ ]:
%%bash --bg
KUBE_EDITOR="code -w" kubectl -n news-api edit rolebinding/news-api

As Alice, obtain an access token from the Keycloak server:

In [ ]:
%%bash --out ACCESS_TOKEN
curl http://keycloak.127.0.0.1.nip.io/realms/devconf/protocol/openid-connect/token -s -d 'grant_type=password' -d 'client_id=demo' -d 'username=alice' -d 'password=p' | jq -r .access_token

Try the News API as Alice:

In [ ]:
!curl -H "Authorization: Bearer $ACCESS_TOKEN" http://news-api.127.0.0.1.nip.io/sports -i

As Bob, obtain an access token from the Keycloak server:

In [ ]:
%%bash --out ACCESS_TOKEN
curl http://keycloak.127.0.0.1.nip.io/realms/devconf/protocol/openid-connect/token -s -d 'grant_type=password' -d 'client_id=demo' -d 'username=bob' -d 'password=p' | jq -r .access_token

Try the News API as Bob:

In [ ]:
!curl -H "Authorization: Bearer $ACCESS_TOKEN" http://news-api.127.0.0.1.nip.io/sports -i

### 5. Inject auth data in the request

Modify the AuthConfig:

In [ ]:
!kubectl -n news-api apply -f authconfig-3.yaml

Create an article with author:

In [ ]:
!curl -H 'Authorization: APIKEY ndyBzreUzF4zqDQsqSPMHkRhriEOtcRx' -X POST \
  -d '{"title":"Biden to sign massive climate, health care legislation","body":"President Joe Biden will sign Democrats’ landmark climate change and health care bill on Tuesday, delivering what he has called the “final piece” of his pared-down domestic agenda, as he aims to boost his party’s standing with voters less than three months before midterm elections. (By The Associated Press)"}' \
  http://news-api.127.0.0.1.nip.io/politics

List the news articles in the 'politics' category:

In [ ]:
!curl -H 'Authorization: APIKEY ndyBzreUzF4zqDQsqSPMHkRhriEOtcRx' http://news-api.127.0.0.1.nip.io/politics

## Cleanup

In [ ]:
!kind delete cluster --name authorino-demo